In [1]:
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from IPython.display import display 
from sklearn.metrics import fbeta_score
import model_utils as model_utils
from sklearn.externals import joblib

import json
# Pretty display for notebooks
%matplotlib inline
pd.set_option('display.max_colwidth', -1)


with open('best_config.json') as json_data_file:
    config = json.load(json_data_file)[0]
print(config)
seed = int(config['seed'])
test_size = config['test_size']

dataset = pd.read_csv("datasets/experiments/cie10.csv")
display(dataset.head(n = 5))

labels = dataset[['RIESGO_VIDA']]
features = dataset.drop(['RIESGO_VIDA'], axis = 1)


C:\Users\mrugeles\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


{'seed': 134.0, 'test_size': 0.1, 'impute': 0.0}


,AFEC_DPTO,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_MPIO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,ALTO_COSTO,...,PQR_TIPOPETICION,TRIM,CAPITULO,COD_CIE_10_03,COD_CIE_10_04,CIE10_SEXO,AFEC_EDADR_INF,AFEC_EDADR_SUP,CIE10_RANGO_EDAD,RIESGO_VIDA
0,0.425153,0.400555,0.458335,0.42859,0.454756,0.537595,0.532278,0.419117,0.431765,0.426038,...,0.455397,0.405202,0.420753,0.659895,0.795501,0.407151,0.436994,0.436994,0.413772,1
1,0.552813,0.436647,0.458335,0.42859,0.557585,0.537595,0.419145,0.419117,0.431765,0.426038,...,0.455397,0.405202,0.370675,0.203957,0.111111,0.407151,0.436994,0.436994,0.413772,0
2,0.491303,0.435185,0.458335,0.42859,0.485267,0.537595,0.419145,0.380646,0.431765,0.426038,...,0.455397,0.405202,0.453680,0.457447,0.466667,0.407151,0.436994,0.436994,0.413772,0
3,0.552813,0.400555,0.412468,0.42859,0.527363,0.426399,0.532278,0.419117,0.431765,0.426038,...,0.455397,0.405202,0.248912,0.529412,0.727259,0.407151,0.435873,0.435873,0.413772,1
4,0.411579,0.400555,0.458335,0.42859,0.401531,0.383166,0.419145,0.419117,0.431765,0.426038,...,0.455397,0.405202,0.311155,0.134942,0.130781,0.407151,0.435873,0.435873,0.413772,0


In [11]:
 pd.read_csv("datasets/experiments/cie10.csv").head()

,AFEC_DPTO,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_MPIO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,ALTO_COSTO,...,PQR_TIPOPETICION,TRIM,CAPITULO,COD_CIE_10_03,COD_CIE_10_04,CIE10_SEXO,AFEC_EDADR_INF,AFEC_EDADR_SUP,CIE10_RANGO_EDAD,RIESGO_VIDA
0,0.425153,0.400555,0.458335,0.42859,0.454756,0.537595,0.532278,0.419117,0.431765,0.426038,...,0.455397,0.405202,0.420753,0.659895,0.795501,0.407151,0.436994,0.436994,0.413772,1
1,0.552813,0.436647,0.458335,0.42859,0.557585,0.537595,0.419145,0.419117,0.431765,0.426038,...,0.455397,0.405202,0.370675,0.203957,0.111111,0.407151,0.436994,0.436994,0.413772,0
2,0.491303,0.435185,0.458335,0.42859,0.485267,0.537595,0.419145,0.380646,0.431765,0.426038,...,0.455397,0.405202,0.453680,0.457447,0.466667,0.407151,0.436994,0.436994,0.413772,0
3,0.552813,0.400555,0.412468,0.42859,0.527363,0.426399,0.532278,0.419117,0.431765,0.426038,...,0.455397,0.405202,0.248912,0.529412,0.727259,0.407151,0.435873,0.435873,0.413772,1
4,0.411579,0.400555,0.458335,0.42859,0.401531,0.383166,0.419145,0.419117,0.431765,0.426038,...,0.455397,0.405202,0.311155,0.134942,0.130781,0.407151,0.435873,0.435873,0.413772,0


# Experiment results

In [2]:
import pandas as pd
import experiment_datasets as datax
from IPython.display import display, Markdown, Latex

dataset = pd.read_csv(f'datasets/experiments/cie10_results.csv')
display(dataset)

,learner,train_time,pred_time,f_test,f_train
0,SGDClassifier,0.219661,0.007472,0.833008,0.830589
1,MLPClassifier,275.244034,1.771611,0.831914,0.835376
2,PassiveAggressiveClassifier,0.225209,0.007232,0.823599,0.819058
3,XGBClassifier,22.300595,0.524287,0.815864,0.814867
4,GradientBoostingClassifier,22.718659,0.320232,0.809177,0.809231
5,RandomForestClassifier,2.571832,0.437573,0.790145,0.983948
6,BaggingClassifier,13.959082,0.620450,0.783342,0.982915
7,AdaBoostClassifier,5.844942,0.555662,0.782806,0.780315
8,LogisticRegression,2.526419,0.007063,0.781326,0.778550
9,ExtraTreesClassifier,2.488391,0.557214,0.780588,0.998980


In [3]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'labels' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = test_size, random_state = seed, stratify=labels)

# Show the results of the split
print("features_final set has {} samples.".format(features.shape[0]))
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))


features_final set has 175360 samples.
Training set has 157824 samples.
Testing set has 17536 samples.


# Tuning Models

## Tuning SGDClassifier

In [ ]:
from sklearn.metrics import make_scorer
import xgboost as xgb

sdg_classifier = xgb.XGBClassifier()
print(sdg_classifier)

sdg_parameters = { 
  'max_depth':[15],
  'n_estimators':[100, 150],
  #'booster': ['gbtree', 'gblinear', 'dart'],
  'n_jobs':[16]
}

sdg_classifier, default_sdg_score, tunned_sdg_score, sdg_matrix = model_utils.tune_classifier(sdg_classifier, sdg_parameters, X_train, X_test, y_train, y_test)

model_utils.plot_confusion_matrix(sdg_matrix, classes=['Life not as risk', 'Life at risk'], normalize = True)
print("Unoptimized model\n------")
print("F-score: {:.4f}".format(default_sdg_score))
print("\nOptimized Model\n------")
print("F-score: {:.4f}".format(tunned_sdg_score))
joblib.dump(sdg_classifier, 'sdg_classifier.joblib')
print(sdg_classifier)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


C:\Users\mrugeles\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


## Tuning AdaBoostClassifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_classifier = AdaBoostClassifier(random_state = seed)

adaParameters = {
  'learning_rate':[0.1, 0.5, 1],
  'algorithm' :['SAMME', 'SAMME.R']
}

ada_classifier, default_ada_score, tuned_ada_score, cnf_ada_matrix = model_utils.tune_classifier(ada_classifier, adaParameters,  X_train, X_test, y_train, y_test)

model_utils.plot_confusion_matrix(cnf_ada_matrix, classes=['Life not as risk', 'Life at risk'], normalize = True)
print("Unoptimized model\n------")
print("F-score on testing data: {:.4f}".format(default_ada_score))
print("\nOptimized Model\n------")
print("Final F-score on the testing data: {:.4f}".format(tuned_ada_score))

joblib.dump(ada_classifier, 'ada_classifier.joblib') 


## Tuning GaussianClassifier


In [ ]:
gauss_classifier = GaussianNB()

gaussParameters = {
  'priors':[None],
  'var_smoothing' :[1e-09]
}

gauss_classifier, default_gauss_score, tuned_gauss_score, cnf_gauss_matrix = model_utils.tune_classifier(gauss_classifier, gaussParameters,  X_train, X_test, y_train, y_test)

model_utils.plot_confusion_matrix(cnf_gauss_matrix, classes=['Life not as risk', 'Life at risk'], normalize = True)
print("Unoptimized model\n------"))
print("F-score on testing data: {:.4f}".format(default_gauss_score))
print("\nOptimized Model\n------")
print("Final F-score on the testing data: {:.4f}".format(tuned_gauss_score))

joblib.dump(gauss_classifier, 'gauss_classifier.joblib') 


# Stacking
## Stacking with to best classifiers

In [ ]:
sclf_two, sclf_score = model_utils.get_stack_two(rf_classifier, ada_classifier, X_train, X_test, y_train, y_test, seed)

joblib.dump(sclf_two, 'sclf_two.joblib') 

model_utils.model_validation('sclf_two.joblib', X_test, y_test)

## Stacking with all classifiers

In [ ]:
sclf_all, sclf_all_score = model_utils.get_stack_all(rf_classifier, ada_classifier, gauss_classifier, X_train, X_test, y_train, y_test, seed)

joblib.dump(sclf_all, 'sclf_all.joblib') 

model_utils.model_validation('sclf_all.joblib', X_test, y_test)